### Hausarbeit 8

• Modifizieren Sie das ResNet Beispiel aus dem Buch so, dass es dem
Netzwerk aus dem Paper “Wide Residual Networks” von Zagoruyko et al.
entspricht <br>
• Verwenden Sie die gleichen Augmentation Methods wie in dem Paper
(RandomHorizontalFlip und RandomCrop)<br>
• Vergleichen Sie die Accuracy (Erkennungsraten) für Trainings- und
Validierungsdaten mit dem Beispiel aus dem Buch<br>
• Das Netzwerk dabei nur für die Klassifikation von Vögeln und Flugzeugen
trainieren.<br>
• Verwenden Sie 28 B(3,3) Layer mit k=2, [WRN-28-2-B(3,3)]<br>

#### 1.Schritt: Laden der notwendigen Bibliotheken

In [27]:
# Laden der notwendigen Bibliotheken
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
import collections
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import datetime
import torch
import time
import random

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

device = torch.device('cuda')

#### 2.Schritt: Laden der Bilddaten aus dem CIFAR10-Datensatz. Klassen: Vögel und Flugzeuge

In [28]:
# Definition der einzelnen Klassen
class_names_bird_plane = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

# Laden des Datensatzes: Umwandlung in einen Tensor und Normalisierung der Daten
data_path = '../data-unversioned/p1ch6/'

# Trainingsdatensatz
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32),
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                     (0.2470, 0.2435, 0.2616))
    ]))

# Validierungsdatensatz
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32),
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Bildung eines neuen Datensatzes, nur mit Bird und Flugzeug Bildern
label_map = {0: 0, 2: 1}
class_names_bird_plane = ['airplane', 'bird']

# Trainingsdatensatz
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

# Validierungsdatensatz
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

Files already downloaded and verified
Files already downloaded and verified


#### 3.Schritt: Definition des ursprünglichen Modells mit dem 3x3-Kernel (Residual-Block aus dem Buch):


In [40]:
# Definition Residual-Block für ein neuronales Netzwerk
class ResBlock(nn.Module):
    def __init__(self, n_chans):

        # Zur Initialisierung des RES-Blocks Pbjekts wird ein Konstruktor gebildet.
        super(ResBlock, self).__init__()
        # 2D-Faltungsoperation mit einer 3x3 Kernel-Größe und einem Padding von 1.
        # n_chans = Eingangskanäle und Ausgangskanäle
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                              padding=1, bias=False)
        # Definition einer Batch-Normalisierung, die auf n_features angewendet wird.
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        # Gewichtung der Faltung mit der Kaiming-Normalisierung --> Abgestimmt auf die ReLU-Aktivierungsfunktion
        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')
        # Gewichte der Batch-Normalisierung werden auf den Wert 0.5 gesetzt.
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        # Biases der Batch-Normalisierung werden gleich 0 gesetzt.
        torch.nn.init.zeros_(self.batch_norm.bias)

# Definition der Forward-Methode
    def forward(self, x):

        # 1. Layer
        # Der Eingabetensor x wird durch die Faltung (self.conv) geleitet,
        # um eine Zwischenausgabe out zu erhalten.
        out = self.conv(x)
        # 2. Layer
        # Die Zwischenausgabe out wird durch die Batch-Normalisierung
        # (self.batch_norm) geleitet, um normierte Features zu erhalten.
        out = self.batch_norm(out)
        # 3. Layer
        # Die normierten Features werden durch die ReLU-Aktivierungsfunktion
        # geleitet (torch.relu).
        out = torch.relu(out)
        # Die Zwischenausgabe out wird mit dem Eingabetensor x addiert,
        # um den Residual-Verbindung zu implementieren.
        return out + x

# Definition des neuronalen Netzwerks mit Residualblöcken
class NetResDeep(nn.Module):

    def __init__(self, n_chans1=32, n_blocks=10):

        # Konstrukur, um das NetResDeep-Objekt zu initialisieren
        super().__init__()
        # Festlegung der Kanäle (Channels), die in der ersten Faltungsschicht verwendet werden sollen.
        # Hier = 32
        self.n_chans1 = n_chans1
        # Definition einer 2D-Faltungsschicht, die einen Eingabetensor mit 3 Kanälen (RGB-Bild) nimmt und
        # n_channel Ausgangskanäle erzeugt.
        # Kernel: 3x3 und Padding: 1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        # Definition einer Sequenz von Residual-Blöcken
        # Die Sequenz: es wird n_blocks-mal ein Residualblock mit n_chans1 Kanälen
        # erstellt und in die Sequenz eingefügt.
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        # Definition eines vollständig verbundenen Layers
        # Unter der Annahme, dass der Input nach dem Pooling 8x8 groß ist.
        # Ausgabevektor ist danach 32-Groß
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        # Definition zweiter vollständig verbundener Layer
        # Eingabevektor = 32 und Ausgabevektor = 2
        self.fc2 = nn.Linear(32, 2)

# Definition der Forward-Methode
    def forward(self, x):
        # Der Eingabetensor x wird durch die erste Faltung (self.conv1)
        # geleitet und dann mit der ReLU-Aktivierungsfunktion und einer
        # 2x2-Max-Pooling-Operation verarbeitet.
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        # Die verarbeiteten Features werden durch die Sequenz von
        # Residualblöcken (self.resblocks) geleitet.
        out = self.resblocks(out)
        # Die resultierenden Features werden erneut mit einer
        # 2x2-Max-Pooling-Operation verarbeitet.
        out = F.max_pool2d(out, 2)
        # Die resultierenden Features werden in einen Vektor umgeformt
        # (out.view(-1, 8 * 8 * self.n_chans1)), um sie für
        # den vollständig verbundenen Layer vorzubereiten.
        out = out.view(-1, 8 * 8 * self.n_chans1)
        # Der umgeformte Vektor wird durch den ersten vollständig verbundenen
        # Layer (self.fc1) mit ReLU-Aktivierung geleitet.
        out = torch.relu(self.fc1(out))
        # Die resultierenden Features werden durch den zweiten vollständig
        # verbundenen Layer (self.fc2) geleitet, um den endgültigen Ausgabevektor zu erhalten.
        out = self.fc2(out)
        # Der Ausgabevektor wird zurückgegeben.
        return out

# Definition des Trainingloops
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

# Trainieren des Modells
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)

model_Net_Res = NetResDeep(n_chans1=32, n_blocks=28).to(device=device) # Model
optimizer = optim.SGD(model_Net_Res.parameters(), lr=1e-2) # Optimizer
loss_fn = nn.CrossEntropyLoss()  # Loss-Funktion

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model_Net_Res,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

# Berechnung der Genauigkeit des Modells für die Trainings und Validierungsdaten:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))


validate(model_Net_Res, train_loader, val_loader)
print("Number of free parameters in the model:",sum(p.numel() for p in model_Net_Res.parameters()))

2023-06-19 12:57:48.908705 Epoch 1, Training loss 1.11557389786289
2023-06-19 12:58:04.787877 Epoch 10, Training loss 0.2743010007936484
2023-06-19 12:58:22.389566 Epoch 20, Training loss 0.1885548646852469
2023-06-19 12:58:39.786741 Epoch 30, Training loss 0.11330985504493213
2023-06-19 12:58:57.433600 Epoch 40, Training loss 0.06318548567308362
2023-06-19 12:59:15.020603 Epoch 50, Training loss 0.09134652771820925
2023-06-19 12:59:32.397530 Epoch 60, Training loss 0.06883334765614123
2023-06-19 12:59:49.703252 Epoch 70, Training loss 0.014727749402603657
2023-06-19 13:00:07.363924 Epoch 80, Training loss 0.014398649451693602
2023-06-19 13:00:24.570629 Epoch 90, Training loss 0.0889885771224738
2023-06-19 13:00:41.975524 Epoch 100, Training loss 0.04143749161736126
Accuracy train: 0.87
Accuracy val: 0.79
Number of free parameters in the model: 75810


#### 4.Schritt: WRN: Wide Residual Network

**Anforderungen an das Netzwerk:**
- Augmentation Methoden: RandomHorizontalFlip und RandomCrop

- Sie 28 B(3,3) Layers mit k=2, [WRN-28-2-B(3,3)]


In [37]:
# Definition des Residual-Blocks für das neuronale Netzwerk
class WRN_Block(nn.Module):
    # Übergabe der Anzahl der Channels und des Strides
    def __init__(self, in_channels, out_channels, stride=1):
        super(WRN_Block, self).__init__()
        # Batch-Normalisierung der Anzahl der in_channels
        self.bn1 = nn.BatchNorm2d(in_channels)
        # ReLU-Aktivierungsfunktion
        self.relu1 = nn.ReLU(inplace=True)
        # 1. Convolutional-Layer im Block.
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        # Batch-Normalisierung definiert mit der Anzahl der out_channels
        self.bn2 = nn.BatchNorm2d(out_channels)
        # RelU-Aktivierungsfunktion
        self.relu2 = nn.ReLU(inplace=True)
        # 2. Convolutional-Layer im Block
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        # Anpassung, falls in_channels != out_channels
        self.adjust = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),)

        torch.nn.init.kaiming_normal_(self.conv1.weight,
                                      nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.conv2.weight,
                                      nonlinearity='relu')

        torch.nn.init.constant_(self.bn1.weight, 0.5)
        torch.nn.init.constant_(self.bn2.weight, 0.5)

        torch.nn.init.constant_(self.bn1.bias, 0.0)
        torch.nn.init.constant_(self.bn2.bias, 0.0)

    # Definition der Forward-Methode
    def forward(self, input):
        # Anwendung der ersten Batch-Normalisierung
        out = self.bn1(input)
        # Anwendung der ersten ReLU-Aktivierungsfunktion
        out = self.relu1(out)
        # Anwendung des ersten Convolutional Layers
        out = self.conv1(out)
        # Anwendung der zweiten Batch-Normalisierung
        out = self.bn2(out)
        # Anwendung der zweiten ReLU-Aktivierungsfuntkion
        out = self.relu2(out)
        # Anwendung des zweiten Convolutional Layers
        out = self.conv2(out)
        # Anpassung der Eingabe, sodass zwischen den Blöcken keine Probleme mit unterschiedlich großen Eingabewerten entsteht.
        if input.size() != out.size():
            # Anwendung des zusätzlichen Convolutional Layers, falls die Dimensionen zwischen den Blöcken nicht passen.
            input = self.adjust(input)

        return out + input

# Definition des künstlichen neuronalen Netzes
class WRN(nn.Module):
    # Übergabe der Anzahl der Blöcke, der Weite der Layers und die Anzahl der vorhandenen Klassen
    def __init__(self, num_blocks=4, k=2, num_classes=2):
        super(WRN, self).__init__()
        # Anzahl der input-Channels
        self.in_channels = 16
        # Definition des ersten Convolutional-Layers vor den Residual-Blöcken (convolutional group 1: 16)
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        # Definition der ReLU-Aktivierungsfunktion
        self.relu = nn.ReLU(inplace=True)

        ### Definition der Übergabeparameter für die Blöcke
        # Convolutional group 2: 16*k, num_blocks: Anzahl von Blöcken innerhalb der Gruppe.
        self.group_conv2 = self.block(16*k, num_blocks)
        # Convolutional group 3: 32*k
        self.group_conv3 = self.block(32*k, num_blocks, stride=2)
        # Convolutional group 4: 64*k
        self.group_conv4 = self.block(64*k, num_blocks, stride=2)

        # Definition der Batch-Normalisierung
        self.bn = nn.BatchNorm2d(64*k)
        # Definition des Average Poolings
        self.avg_pool = nn.AvgPool2d(8, stride=1)
        # Definition des Linearisierungsfunktion, mit 64*k Neuronen, die zu num_classes = 2 transferiert werden.
        self.fc = nn.Linear(64*k, num_classes)

    # Erzeugung einer Sequenz aus mehreren (num_blocks) Residual Blöcken
    # Übergabe der Anzahl der Output-Channels und der Anzahl der N-Residual Blöcken
    def block(self, out_channels, num_blocks=4, stride=1):
        # Leere Liste für die einzelnen Residual-Blöcke
        sequence_of_blocks = []
        sequence_of_blocks.append(WRN_Block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            sequence_of_blocks.append(WRN_Block(out_channels, out_channels))
        return nn.Sequential(*sequence_of_blocks)

    def forward(self, x):

        input = x
        # 1. Convolutional-Layer vor den Blöcken
        out_group_conv1 = self.conv1(input)
        # 2. Gruppe: Conv2 - 16*k
        out_group_conv2 = self.group_conv2(out_group_conv1)
        # 3. Gruppe: Conv3 - 32*k
        out_group_conv3 = self.group_conv3(out_group_conv2)
        # 4. Gruppe: Conv4 - 64*k
        out_group_conv4 = self.group_conv4(out_group_conv3)

        # Anwendung der Batch-Normalisierung auf das Ergebnis nach den Blöcken
        out = self.bn(out_group_conv4)
        # Anwendung der ReLU-Aktivierungsfunktion
        out = self.relu(out)
        # Anwendung des Avg-Poolings
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        # Anwendung der Linearisierungsfunktion
        out = self.fc(out)
        return out

# Definition des Training-Loops
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

# Trainieren des Modells
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = WRN().to(device=device) # Modell
lr = 3e-1 # Lernrate
optimizer = optim.SGD(model.parameters(), lr=lr) # Optimizer
loss_fn = nn.CrossEntropyLoss() # Loss-Funktion

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

# Berechnung der Genauigkeit des Modells für die Trainings und Validierungsdaten:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)
print("Number of free parameters in the model:",sum(p.numel() for p in model.parameters()))

2023-06-19 12:46:28.553358 Epoch 1, Training loss 0.4513682360482064
2023-06-19 12:46:48.517420 Epoch 10, Training loss 0.12106433641284135
2023-06-19 12:47:10.847435 Epoch 20, Training loss 0.021550760945159893
2023-06-19 12:47:33.593666 Epoch 30, Training loss 0.0017710104834949775
2023-06-19 12:47:55.581966 Epoch 40, Training loss 0.00015372756504826137
2023-06-19 12:48:17.640569 Epoch 50, Training loss 0.0017521342471728373
2023-06-19 12:48:39.683709 Epoch 60, Training loss 0.00024295221881367782
2023-06-19 12:49:01.705011 Epoch 70, Training loss 0.0013621689838036201
2023-06-19 12:49:23.776445 Epoch 80, Training loss 0.0002636750655748439
2023-06-19 12:49:45.832560 Epoch 90, Training loss 0.00021758072122045945
2023-06-19 12:50:07.934912 Epoch 100, Training loss 0.00021671975004583898
Accuracy train: 1.00
Accuracy val: 0.93
Number of free parameters in the model: 1531090


#### Vergleich der Erkennungsraten: (NetResDeep und WRN)

Es ist erkenntlich, dass die Genauigkeit des Modells durch die Verwendung eines Wide Residual Netzwerks verbessert werden kann.
Das NetResDeep-Model, welches eine Tiefe von 28 besitzt und lediglich einzelne Res-Blöcke hintereinander verwendet, besitzt eine Genauigkeit auf die Validierungsdaten von 79 %.
Dahingegen besitzt das Wide Residual Netzwerk, welches auf der Grundlage des Papers (https://arxiv.org/abs/1605.07146) implementiert wurde, eine Genauigkeit von 93 % auf die Validierungsdaten und ist folglich deutlich präziser. Hierbei besteht das Modell aus 28 Layern, aufgeteilt in vier Blöcke mit einer Weite von k = 2.